In [ ]:
# Importing Libraries
import pandas as pd  # Pandas library for data manipulation
import geopandas as gpd  # GeoPandas library for spatial data handling
from dbfread import DBF  # Library for reading .dbf files
import shapely.ops as so
import shapely.geometry as sg  # Libraries for spatial operations and geometry
import heapq  # Library for heap data structure

# Loading Data
# You can get free download from Korea https://www.its.go.kr/nodelink/nodelinkRef
CWD = './MOCT_DATA/'
ROAD = gpd.read_file(CWD + "/MOCT_link.shp").to_crs(5181)  # Loading and transforming road data to PCS(EPSG:5181)
ROAD['geometry_list'] = [list(x.coords) for x in ROAD['geometry']]
ROAD['geometry_list'] = ROAD['geometry_list'].astype('str')  # Converting geometry for optimization
NODE = gpd.read_file(CWD + "/MOCT_node.shp").to_crs(5181)  # Loading and transforming node data to PCS(EPSG:5181)
TURN = pd.DataFrame(DBF(CWD + "/TURNINFO.dbf"))  # Loading Turninfo data into a DataFrame

# Define Key Functions
# Function to initialize the starting point with longitude and latitude coordinates
def init_set_starting_point(lon, lat):
    # Create a point in EPSG:4326
    point_4326 = sg.Point(127.1208141, 37.6452898)
    # Convert point to EPSG:5181
    point_gdf = gpd.GeoSeries(point_4326).set_crs(4326).to_crs(5181)
    point = point_gdf.set_crs(5181)[0]
    return point

# Function to find the nearest road (Link) from the starting point
def init_find_nearest_road(road, st_point):
    global FLind_init, FLID_init, FNID_init, TNID_init, NP_init
    road_list = road.geometry.to_list()
    c1 = min(road_list, key=st_point.distance)
    NP_init, SP_init = so.nearest_points(c1, st_point)
    d1 = NP_init.distance(SP_init)
    FLind_init = road[road.geometry_list.isin([str(list(c1.coords))])]
    FLind_init = FLind_init[['LINK_ID', 'F_NODE', 'T_NODE', 'MAX_SPD', 'LENGTH', 'geometry', 'geometry_list']].copy()
    FLID_init = FLind_init['LINK_ID'].iloc[0]
    FNID_init = FLind_init['F_NODE'].iloc[0]
    TNID_init = FLind_init['T_NODE'].iloc[0]
    return NP_init

# Other functions for geometry and data manipulation

# Define TURN_X and TURN_O for turn type filtering

# Function to filter turn options based on the starting link, starting node, and target node
def turn_filter(FLID, FNID, TNID):
    global TLind_list
    FLID, FNID, TNID = str(FLID), str(FNID), str(TNID)
    turninfo_df = turn_info(TNID, FLID)
    TLind_list = ROAD[(ROAD['F_NODE'] == TNID) & (ROAD['T_NODE'] != FNID)]
    if len(turninfo_df) > 0:
        for num, turntype in enumerate(turninfo_df['TURN_TYPE']):
            TL_info = turninfo_df[turninfo_df['TURN_TYPE'] == turntype]['ED_LINK'].iloc[0]
            if turntype == "011":
                TLind_list = pd.concat([TLind_list, ROAD[(ROAD['LINK_ID'] == turn_U_TLID(TNID, FLID))]])
            elif turntype in list(TURN_X):
                TLind_list.drop(TLind_list[TLind_list['LINK_ID'] == TL_info].index, inplace=True)
    return TLind_list

# Function to calculate time-based Dijkstra algorithm by nodes

# Time limit for travel
time_limit = float(5)
NODE['DISTANCE'] = float(1e3)

# Set the starting point
point = init_set_starting_point(127.1208141, 37.6452898)

# Calculate the initial distance and cost
init_distance = distance_bw_nodes(init_find_nearest_road(ROAD, point), geometry_node(TNID_init))
init_cost = calc_time(init_distance, speed_link(FLID_init))
NODE['DISTANCE'][NODE[NODE['NODE_ID'] == TNID_init].index[0]] = init_cost
NODE['DISTANCE'][NODE[NODE['NODE_ID'] == FNID_init].index[0]] = float(0)
CLID = {"FNID": FNID_init, "FLID": FLID_init, "TNID": TNID_init}

# Initialize the queue
q = []
for _, i in turn_filter(FLID_init, FNID_init, TNID_init).iterrows():
    CLID = {"FNID": i.F_NODE, "FLID": i.LINK_ID, "TNID": i.T_NODE}
    cost = calc_time(i.LENGTH, i.MAX_SPD) + init_cost
    heapq.heappush(q, (cost, CLID))

st = time.time()

# Main loop for Dijkstra algorithm
while q:
    cost, node = heapq.heappop(q)
    if cost_node(node['TNID']) < cost:
        continue
    for _, j in turn_filter(node['FNID'], node['FLID'], node['TNID']).iterrows():
        CLID = {"FNID": j.F_NODE, "FLID": j.LINK_ID, "TNID": j.T_NODE}
        cost = calc_time(j.LENGTH, j.MAX_SPD) + cost_node(node['FNID'])
        if cost < time_limit and cost < cost_node(j.T_NODE):
            NODE['DISTANCE'][NODE[NODE['NODE_ID'] == node['TNID']].index[0]] = cost
            heapq.heappush(q, (cost, CLID))

ed = time.time()
print(ed - st)
